In [1]:
import pandas as pd

DATA = '/kaggle/input/phl-exoplanet-catalog/phl_exoplanet_catalog_2019.csv'

df = pd.read_csv(filepath_or_buffer=DATA, parse_dates=['P_UPDATED'])
df.head()

,P_NAME,P_STATUS,P_MASS,P_MASS_ERROR_MIN,P_MASS_ERROR_MAX,P_RADIUS,P_RADIUS_ERROR_MIN,P_RADIUS_ERROR_MAX,P_YEAR,P_UPDATED,...,P_HABZONE_CON,P_TYPE_TEMP,P_HABITABLE,P_ESI,S_CONSTELLATION,S_CONSTELLATION_ABR,S_CONSTELLATION_ENG,P_RADIUS_EST,P_MASS_EST,P_SEMI_MAJOR_AXIS_EST
0,11 Com b,3.0,6165.86330,-476.74200,476.74200,NaN,NaN,NaN,2007,2014-05-14,...,0,Hot,0,0.083813,Coma Berenices,Com,Berenice's Hair,12.082709,6165.86330,1.29
1,11 UMi b,3.0,4684.78480,-794.57001,794.57001,NaN,NaN,NaN,2009,2018-09-06,...,0,Hot,0,0.082414,Ursa Minor,UMi,Little Bear,12.229641,4684.78480,1.53
2,14 And b,3.0,1525.57440,NaN,NaN,NaN,NaN,NaN,2008,2014-05-14,...,0,Hot,0,0.081917,Andromeda,And,Andromeda,12.848516,1525.57440,0.83
3,14 Her b,3.0,1481.07850,-47.67420,47.67420,NaN,NaN,NaN,2002,2018-09-06,...,0,Cold,0,0.145241,Hercules,Her,Hercules,12.865261,1481.07850,2.93
4,16 Cyg B b,3.0,565.73385,-25.42624,25.42624,NaN,NaN,NaN,1996,2018-09-06,...,1,Warm,0,0.368627,Cygnus,Cyg,Swan,13.421749,565.73385,1.66


In [2]:
df.shape

(4048, 112)

This is a relatively early dataset (from 2019) and a lot of new exoplanets have been confirmed since then, but we have a lot of data per planet.

What percentage of planets are in their host's habitable zone?

In [3]:
import warnings
from plotly import express

warnings.filterwarnings(action='ignore', category=FutureWarning)

express.pie(data_frame=df['P_HABZONE_CON'].map({0:False, 1:True}).value_counts(normalize=False).to_frame().reset_index(), names='P_HABZONE_CON', values='count', color='P_HABZONE_CON')

What percentage of planets are habitable?

In [4]:
express.pie(data_frame=df['P_HABITABLE'].map({0:False, 1:True}).value_counts(normalize=False).to_frame().reset_index(), names='P_HABITABLE', values='count', color='P_HABITABLE')